# Fine-tune ResNet50 on AffectNet (class 7 excluded)
This notebook:
- Loads AffectNet data organized into folders `0` to `6`
- Applies data augmentation and class balancing
- Fine-tunes ResNet50 using PyTorch
- Tracks loss and accuracy per epoch


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
SAVE_PATH = '/content/drive/MyDrive/resnet50_affectnet.pth'
!unzip -q "/content/drive/MyDrive/datasets.zip" -d "/content/datasets"

Mounted at /content/drive


In [ ]:
import os
import shutil
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.optim as optim
from sklearn.utils.class_weight import compute_class_weight
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from tqdm import tqdm
import time

In [ ]:
if os.path.exists('/content/datasets/separated_dataset/7'):
    os.makedirs('/content/data_excluded', exist_ok=True)
    shutil.move('/content/datasets/separated_dataset/7', '/content/data_excluded/7')

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.2, 0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

data_path = '/content/datasets/separated_dataset'
dataset = datasets.ImageFolder(root=data_path, transform=transform)
loader = DataLoader(dataset, batch_size=64, shuffle=True)

# Compute class weights
targets = [label for _, label in dataset.samples]
num_classes = len(dataset.classes)
class_weights = compute_class_weight('balanced', classes=np.arange(num_classes), y=targets)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).cuda()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
epochs = 3
model.train()
for epoch in range(epochs):
    start_time = time.time()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(loader, desc=f"Epoch {epoch+1}/{epochs}"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    elapsed = time.time() - start_time
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}, Accuracy = {epoch_acc:.4f}, Time = {elapsed:.2f}s")

Epoch 1/3: 100%|██████████| 4436/4436 [59:53<00:00,  1.23it/s]


Epoch 1: Loss = 1.1245, Accuracy = 0.6637, Time = 3593.67s


Epoch 2/3: 100%|██████████| 4436/4436 [58:44<00:00,  1.26it/s]


Epoch 2: Loss = 0.9845, Accuracy = 0.6990, Time = 3524.20s


Epoch 3/3: 100%|██████████| 4436/4436 [58:23<00:00,  1.27it/s]

Epoch 3: Loss = 0.9128, Accuracy = 0.7104, Time = 3503.74s


In [ ]:
torch.save(model.state_dict(), SAVE_PATH)
print(f"Model saved to {SAVE_PATH}")

Model saved to /content/drive/MyDrive/resnet50_affectnet.pth
